In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import tensorflow as tf
import keras_tuner as kt
import keras as kr

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN","NAME", 'USE_CASE', 'ORGANIZATION'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,1,0,N,5000,1
1,T3,Independent,C2000,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,1,0,N,5000,0
3,T3,CompanySponsored,C2000,1,10000-24999,N,6692,1
4,T3,Independent,C1000,1,100000-499999,N,142590,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Look at APPLICATION_TYPE value counts for binning
application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = application_df['APPLICATION_TYPE'].value_counts()[application_df['APPLICATION_TYPE'].value_counts() < 500].index.tolist()

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Apps < 500")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3            27037
T4             1542
T6             1216
T5             1173
T19            1065
T8              737
T7              725
T10             528
Apps < 500      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
counts_binning = application_df['CLASSIFICATION'].value_counts()
counts_binning

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts_classifications = counts_binning[counts_binning>1]
counts_classifications

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(application_df['CLASSIFICATION'].value_counts()[application_df['CLASSIFICATION'].value_counts() < 1000].index)

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Classification < 1000")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000                    17326
C2000                     6074
C1200                     4837
Classification < 1000     2261
C3000                     1918
C2100                     1883
Name: CLASSIFICATION, dtype: int64

In [10]:
# Define bins
bins = [0, 10000, 50000, 100000, np.inf]
labels = ['0-10k', '10k-50k', '50k-100k', '100k+']

# Add a new column to the dataframe with the binned values
application_df['ASK_AMT_RANGE'] = pd.cut(application_df['ASK_AMT'], bins=bins, labels=labels)

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df,dtype=float)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Apps < 500,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_RANGE_0-10k,ASK_AMT_RANGE_10k-50k,ASK_AMT_RANGE_50k-100k,ASK_AMT_RANGE_100k+
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


In [12]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Apps < 500',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Classification < 1000', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y',
       'ASK_AMT_RANGE_0-10k', 'ASK_AMT_RANGE_10k-50k',
       'ASK_AMT_RANGE_50k-1

In [13]:
application_df.drop(columns=['ASK_AMT'], inplace=True)
application_df.columns

Index(['STATUS', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Apps < 500',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Classification < 1000', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       'INCOME_AMT_25000-99999', 'INCOME_AMT_50M+', 'INCOME_AMT_5M-10M',
       'SPECIAL_CONSIDERATIONS_N', 'SPECIAL_CONSIDERATIONS_Y',
       'ASK_AMT_RANGE_0-10k', 'ASK_AMT_RANGE_10k-50k',
       'ASK_AMT_RANGE_50k-100k', 'ASK_

In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
def create_model(hp):
    nn_model = tf.keras.models.Sequential()
    activation = hp.Choice('activation', ['relu', 'tanh', 'sigmoid', 'gelu', 'selu', 'elu'])
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
                                        min_value=1,
                                        max_value=80,
                                        step=2), activation=activation, input_dim=len(X_train_scaled[0])))
    for i in range(hp.Int('num_layers', 1, 10)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                        min_value=1,
                                        max_value=80,
                                        step=2), activation=activation))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
    loss = hp.Choice('loss', ['binary_crossentropy', 'mse', 'mae', 'mape', 'msle'])
    optimizer = hp.Choice('optimizer', ['adam', 'rmsprop', 'sgd', 'adagrad', 'adadelta', 'adamax'])
    nn_model.compile(loss=loss, optimizer=optimizer, metrics=["accuracy"])
    
    return nn_model

In [17]:
tuner = kt.Hyperband(create_model, objective='val_accuracy', max_epochs=20, hyperband_iterations=100, directory='my_dir2', project_name='intro_to_kt')

In [18]:
tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))

Trial 474 Complete [00h 00m 25s]
val_accuracy: 0.46682214736938477

Best val_accuracy So Far: 0.7250145673751831
Total elapsed time: 06h 54m 46s

Search: Running Trial #475

Value             |Best Value So Far |Hyperparameter
selu              |elu               |activation
63                |71                |first_units
8                 |10                |num_layers
21                |19                |units_0
mse               |binary_crossent...|loss
adam              |rmsprop           |optimizer
61                |11                |units_1
21                |61                |units_2
57                |3                 |units_3
7                 |51                |units_4
37                |65                |units_5
21                |61                |units_6
9                 |33                |units_7
1                 |15                |units_8
57                |63                |units_9
20                |7                 |tuner/epochs
7                 |3   

KeyboardInterrupt: 

In [19]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'elu',
 'first_units': 71,
 'num_layers': 10,
 'units_0': 19,
 'loss': 'binary_crossentropy',
 'optimizer': 'rmsprop',
 'units_1': 11,
 'units_2': 61,
 'units_3': 3,
 'units_4': 51,
 'units_5': 65,
 'units_6': 61,
 'units_7': 33,
 'units_8': 15,
 'units_9': 63,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 3,
 'tuner/bracket': 2,
 'tuner/round': 1,
 'tuner/trial_id': '0423'}

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  59
hidden_nodes_layer2 = 11
hidden_nodes_layer3 = 25
hidden_nodes_layer4 = 71
hidden_nodes_layer5 = 65
output_dim = 1



nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="tanh"))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="tanh"))
nn_model.add(tf.keras.layers.Dense(units=hidden_nodes_layer5, activation="tanh"))



# Output layer
nn_model.add(tf.keras.layers.Dense(output_dim, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 59)                2773      
                                                                 
 dense_4 (Dense)             (None, 11)                660       
                                                                 
 dense_5 (Dense)             (None, 25)                300       
                                                                 
 dense_6 (Dense)             (None, 71)                1846      
                                                                 
 dense_7 (Dense)             (None, 65)                4680      
                                                                 
 dense_8 (Dense)             (None, 1)                 66        
                                                                 
Total params: 10,325
Trainable params: 10,325
Non-trai

In [ ]:
nn_model.compile(loss="mse", optimizer="sgd", metrics=["accuracy"])

In [ ]:


# Define your boosting parameters
n_boosting_iterations = 5
learning_rate = 0.1

# Initialize weights for each instance in the training data
weights = np.ones(len(X_train_scaled)) / len(X_train_scaled)

# Define the boosting loop
for i in range(n_boosting_iterations):
    # Train the base model on the weighted training set
    nn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    nn_model.fit(X_train_scaled, y_train, sample_weight=weights)
    
    # Evaluate the base model on the validation set
    loss, accuracy = nn_model.evaluate(X_test, y_test)
    
    # Update the weights based on misclassifications
    predictions = nn_model.predict(X_test)
    misclassified_indices = np.argmax(predictions, axis=1) != np.argmax(y_test, axis=1)
    weights[misclassified_indices] *= 1.5  # increase weights for misclassified instances
    
    # Normalize the weights to sum up to 1
    weights /= np.sum(weights)
    
# Combine predictions of the boosted models
boosted_predictions = np.zeros((len(X_test), output_dim))
for _ in range(n_boosting_iterations):
    predictions = nn_model.predict(X_test)
    boosted_predictions += predictions

final_predictions = np.argmax(boosted_predictions, axis=1)


Epoch 1/100
804/804 [==============================] - 5s 5ms/step - loss: 0.2136 - accuracy: 0.6799
Epoch 2/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1995 - accuracy: 0.7125
Epoch 3/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1966 - accuracy: 0.7191
Epoch 4/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1948 - accuracy: 0.7227
Epoch 5/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1934 - accuracy: 0.7251
Epoch 6/100
804/804 [==============================] - 3s 4ms/step - loss: 0.1923 - accuracy: 0.7255
Epoch 7/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1914 - accuracy: 0.7259
Epoch 8/100
804/804 [==============================] - 4s 6ms/step - loss: 0.1905 - accuracy: 0.7266
Epoch 9/100
804/804 [==============================] - 4s 5ms/step - loss: 0.1898 - accuracy: 0.7270
Epoch 10/100
804/804 [==============================] - 4s 4ms/step - loss: 0.1892 - accura

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.1864 - accuracy: 0.7279 - 858ms/epoch - 3ms/step
Loss: 0.18638290464878082, Accuracy: 0.7279300093650818


In [ ]:
# Export our model to HDF5 file
nn_model.save("AlphabetSoupCharity2.h5")